In [1]:
import pandas as pd
import datetime as dt
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

# Consigna

Vamos a trabajar con una base de datos elaborada por [Sysarmy](https://sysarmy.com/es/) (una comunidad de profesionales en el área de sistemas) que contiene los resultados de una encuesta se suedlos realizada en 2020. La encuesta tiene algunso campos tipificados y otros campos de texto libre.

Queremos analizar los resultados de esta encuesta. Nos interesan especialmente aquellas respuestas vinculadas a profecionales que trabajan con datos y que poseen título universitario (para eliminar algunos casos con comportamiento atípico). Los objetivos de esta práctica serán:

1. Construir una variable que nos indique hace cuánto tiempo fue respondida la encuesta.

2. Encontrar el salario promedio para los trabajos vinculados a datos. Puede ser general o abierto por las distintas variantes.

3. Contabilizar cuantos lenguajes de programación maneja la persona.


In [2]:
sueldos = pd.read_csv("https://raw.githubusercontent.com/eea-uba/EEA-2020/master/Fuentes/encuesta_sueldos_sysarmy_1s2020.csv")
sueldos.head()


,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,sos_miembro_de_alguna_comunidad_it,cantidad_de_empleados,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado
0,1/31/2020 6:50:26,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,10,3,1,0,Secundario,...,NaN,10001+,Producto basado en Software,8,10,6,"Capacitaciones y/o cursos, Comidas pagas / sub...",No,cognizant,False
1,1/28/2020 9:27:48,Hombre,30,Argentina,Ciudad Autónoma de Buenos Aires,3,3,3,0,Terciario,...,NaN,51-100,Otras industrias,7,3,35,Capacitaciones y/o cursos,No,mercadolibre,False
2,1/29/2020 16:54:29,Mujer,40,Argentina,Ciudad Autónoma de Buenos Aires,15,3,3,1,Universitario,...,NaN,11-50,Servicios / Consultoría de Software / Digital,8,8,30,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,False
3,2/1/2020 5:46:25,Mujer,36,Argentina,Ciudad Autónoma de Buenos Aires,10,3,3,0,Universitario,...,NaN,51-100,Otras industrias,7,5,4,Viáticos,No,NaN,False
4,2/1/2020 17:51:21,Hombre,29,Argentina,Ciudad Autónoma de Buenos Aires,10,2,2,0,Universitario,...,NaN,11-50,Otras industrias,5,8,7,Ninguna de las anteriores,No,NaN,False


In [3]:
# observamos faltantes y tipos de datos
sueldos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 53 columns):
 #   Column                                                                                          Non-Null Count  Dtype  
---  ------                                                                                          --------------  -----  
 0   timestamp                                                                                       5982 non-null   object 
 1   me_identifico                                                                                   5982 non-null   object 
 2   tengo                                                                                           5982 non-null   int64  
 3   estoy_trabajando_en                                                                             5982 non-null   object 
 4   donde_estas_trabajando                                                                          5982 non-null   object 
 5   anos_de_exper

In [4]:
# Antes de comenzar, filtramos solo los registros con educacion universitaria
condicion = ~(sueldos['nivel_de_estudios_alcanzado'].str.lower().isin(
    ['secundario', 'primario']))
sueldos = sueldos[condicion]
sueldos.head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,sos_miembro_de_alguna_comunidad_it,cantidad_de_empleados,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado
1,1/28/2020 9:27:48,Hombre,30,Argentina,Ciudad Autónoma de Buenos Aires,3,3,3,0,Terciario,...,NaN,51-100,Otras industrias,7,3,35,Capacitaciones y/o cursos,No,mercadolibre,False
2,1/29/2020 16:54:29,Mujer,40,Argentina,Ciudad Autónoma de Buenos Aires,15,3,3,1,Universitario,...,NaN,11-50,Servicios / Consultoría de Software / Digital,8,8,30,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,False
3,2/1/2020 5:46:25,Mujer,36,Argentina,Ciudad Autónoma de Buenos Aires,10,3,3,0,Universitario,...,NaN,51-100,Otras industrias,7,5,4,Viáticos,No,NaN,False
4,2/1/2020 17:51:21,Hombre,29,Argentina,Ciudad Autónoma de Buenos Aires,10,2,2,0,Universitario,...,NaN,11-50,Otras industrias,5,8,7,Ninguna de las anteriores,No,NaN,False
5,2/1/2020 23:13:41,Hombre,27,Argentina,Ciudad Autónoma de Buenos Aires,4,3,2,0,Universitario,...,No,1-10,Otras industrias,2,6,12,"Capacitaciones y/o cursos, Descuentos varios (...",No,NaN,False


## 1) Fecha de encuesta

Pista: la función **datetime.datetime.now()** nos devuelve fecha y hora actuales.

In [5]:
sueldos.timestamp.unique()

array(['1/28/2020 9:27:48', '1/29/2020 16:54:29', '2/1/2020 5:46:25', ...,
       '1/31/2020 13:54:08', '1/31/2020 19:19:38', '2/2/2020 4:45:31'],
      dtype=object)

In [6]:
# convertimos la variable datestamp a date
sueldos['timestamp'] = pd.to_datetime(sueldos['timestamp'],format='mixed')


In [7]:
sueldos["timestamp"]

1      2020-01-28 09:27:48
2      2020-01-29 16:54:29
3      2020-02-01 05:46:25
4      2020-02-01 17:51:21
5      2020-02-01 23:13:41
               ...        
5977   2020-01-30 04:48:47
5978   2020-01-30 11:41:15
5979   2020-01-31 13:54:08
5980   2020-01-31 19:19:38
5981   2020-02-02 04:45:31
Name: timestamp, Length: 5425, dtype: datetime64[ns]

In [8]:
# generamos la variable de antiguedad (diferencia con hoy)
today = dt.datetime.now()
def dias_antiguedad(x):
    return (today - x).days

sueldos['antiguedad'] = sueldos['timestamp'].apply(dias_antiguedad)
sueldos['antiguedad']

1       1606
2       1604
3       1602
4       1601
5       1601
        ... 
5977    1604
5978    1603
5979    1602
5980    1602
5981    1601
Name: antiguedad, Length: 5425, dtype: int64

## 2) Trabajo con datos

In [9]:
sueldos.trabajo_de.unique()

array(['Technical Support', 'Abogada', 'Abogado', 'Account Manager-Sales',
       'Adm', 'Administración', 'Administrador de Aplicaciones',
       'Administrador de Infraestructura',
       'Administrador MS/Exchange/Office365', 'Administrativa',
       'Administrativa contable', 'Administrative Assistant',
       'Administrativo', 'Administrativo de primera en centro de salud',
       'Administrativo en farmacia', 'Advertising', 'Agile Coach',
       'Agile coach',
       'Análisis, diseño, programación, implementación y mantenimiento.',
       'Analista', 'Analista de Metodología y QA',
       'Analista de Monitoreo', 'Analista de Operaciones de Servicios',
       'Analista de prevención de lavado de activos', 'Infosec',
       'Analista de Infosec', 'Analista de Infosec lvl 1',
       'Analista de Sistemas', 'Analista de Soporte',
       'Analista de soporte', 'Analista de Soporte TI ssr',
       'Analista desarrollador Full-stack', 'Analista Funciona ERP',
       'Functional Analys

In [10]:
# llevamos todo a minuscula
sueldos['trabajo_de'] = sueldos.trabajo_de.str.lower()

In [11]:
# creo una variable binaria para ver si trabaja con datos
sueldos['trabaja_con_datos'] = sueldos['trabajo_de'].str.contains('data')

In [12]:
# Verificamos nuestro filtro
sueldos[sueldos['trabaja_con_datos'] == True].trabajo_de.unique()

array(['bi analyst / data analyst', 'data scientist / data engineer',
       'data visualization', 'data visualization engineer',
       'data center operadores de as 400', 'data entry',
       'data entry and management', 'lider data analyst qa - tester'],
      dtype=object)

In [13]:
# Filtramos los profesionales de datos
condicion = (sueldos['trabaja_con_datos'] == True)

sueldos_data = sueldos.loc[condicion]
sueldos_data.head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado,antiguedad,trabaja_con_datos
240,2019-12-20 09:43:06,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,14,6,3,2,Universitario,...,Servicios / Consultoría de Software / Digital,7,6,5,"Clases de idiomas, Gimnasios gratuitos / subve...",No,NaN,True,1645,True
242,2019-12-10 00:42:24,Hombre,28,Argentina,Buenos Aires,2,2,2,1,Universitario,...,Servicios / Consultoría de Software / Digital,6,8,1,"Comidas pagas / subvencionadas, Gimnasios grat...",Sí,NaN,False,1655,True
243,2020-01-13 11:30:32,Hombre,40,Argentina,Ciudad Autónoma de Buenos Aires,17,6,6,0,Posgrado,...,Servicios / Consultoría de Software / Digital,9,1,1,"Horarios flexibles, WFH (Trabajar desde casa)",Sí,NaN,False,1620,True
244,2019-12-10 08:26:52,Hombre,34,Argentina,Formosa,9,5,5,0,Universitario,...,Otras industrias,7,9,45,"Capacitaciones y/o cursos, Comidas pagas / sub...",No,NaN,False,1655,True
245,2020-02-01 18:41:23,Hombre,51,Argentina,Ciudad Autónoma de Buenos Aires,16,8,6,0,Universitario,...,Producto basado en Software,10,6,300,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,True,1601,True


In [14]:
# Normalizamos el puesto de 'data entry' y 'data analyst
sueldos_data['trabajo_de'] = sueldos_data.trabajo_de.str.replace(
    'data entry and management', 'data entry')
sueldos_data['trabajo_de'] = sueldos_data.trabajo_de.str.replace(
    'lider data analyst qa - tester', 'bi analyst / data analyst')

In [15]:
#Vemos los valores unicos
sueldos_data['trabajo_de'].unique()

array(['bi analyst / data analyst', 'data scientist / data engineer',
       'data visualization', 'data visualization engineer',
       'data center operadores de as 400', 'data entry'], dtype=object)

In [16]:
# Ahora vemos el salario promedio
np.mean(sueldos_data.salario_mensual_bruto_en_tu_moneda_local)

150429.5220125786

In [29]:
# Agrupamos por "trabajo_de" obteniendo el promedio y la mediana 
sueldos_data.groupby('trabajo_de').agg({'salario_mensual_bruto_en_tu_moneda_local': ['mean', 'median']})

salario_mensual_bruto_en_tu_moneda_local  \
                                                                     mean   
trabajo_de                                                                  
bi analyst / data analyst                                   130300.127778   
data center operadores de as 400                             32000.000000   
data entry                                                   24000.000000   
data scientist / data engineer                              182261.856061   
data visualization                                          120000.000000   
data visualization engineer                                 100000.000000   

                                            
                                    median  
trabajo_de                                  
bi analyst / data analyst          64000.0  
data center operadores de as 400   32000.0  
data entry                         26000.0  
data scientist / data engineer     90000.0  
data visualization                120000.0  
data visualization engineer       100000.0

## 3) Lenguajes de programación

In [19]:
#Observamos la variable lenguajes_de_programacion
sueldos_data['lenguajes_de_programacion']

240                                    Java, SQL, VBA
242                                               SQL
243                                            Python
244     .NET, C#, CSS, HTML, PHP, Python, R, SQL, VBA
245                  Assembler, C#, COBOL, HTML, Java
                            ...                      
908                                               SQL
910                                       Python, SQL
911                                           C#, SQL
912                                               SQL
3704                          HTML, Java, Python, SQL
Name: lenguajes_de_programacion, Length: 318, dtype: object

In [23]:
#La transformamos en una lista con split
sueldos_data['lenguajes_de_programacion_split'] = sueldos_data['lenguajes_de_programacion'].str.split(',')
sueldos_data['lenguajes_de_programacion_split']

240                                    [Java,  SQL,  VBA]
242                                                 [SQL]
243                                              [Python]
244     [.NET,  C#,  CSS,  HTML,  PHP,  Python,  R,  S...
245                [Assembler,  C#,  COBOL,  HTML,  Java]
                              ...                        
908                                                 [SQL]
910                                        [Python,  SQL]
911                                            [C#,  SQL]
912                                                 [SQL]
3704                         [HTML,  Java,  Python,  SQL]
Name: lenguajes_de_programacion_split, Length: 318, dtype: object

In [21]:
sueldos_data.head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado,antiguedad,trabaja_con_datos,lenguajes_de_programacion_split
240,2019-12-20 09:43:06,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,14,6,3,2,Universitario,...,7,6,5,"Clases de idiomas, Gimnasios gratuitos / subve...",No,NaN,True,1645,True,"[Java, SQL, VBA]"
242,2019-12-10 00:42:24,Hombre,28,Argentina,Buenos Aires,2,2,2,1,Universitario,...,6,8,1,"Comidas pagas / subvencionadas, Gimnasios grat...",Sí,NaN,False,1655,True,[SQL]
243,2020-01-13 11:30:32,Hombre,40,Argentina,Ciudad Autónoma de Buenos Aires,17,6,6,0,Posgrado,...,9,1,1,"Horarios flexibles, WFH (Trabajar desde casa)",Sí,NaN,False,1620,True,[Python]
244,2019-12-10 08:26:52,Hombre,34,Argentina,Formosa,9,5,5,0,Universitario,...,7,9,45,"Capacitaciones y/o cursos, Comidas pagas / sub...",No,NaN,False,1655,True,"[.NET, C#, CSS, HTML, PHP, Python, R, S..."
245,2020-02-01 18:41:23,Hombre,51,Argentina,Ciudad Autónoma de Buenos Aires,16,8,6,0,Universitario,...,10,6,300,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,True,1601,True,"[Assembler, C#, COBOL, HTML, Java]"


In [24]:
#Creamos una variable cantidad_lenguajes  
sueldos_data['cantidad_lenguajes'] = sueldos_data['lenguajes_de_programacion_split'].str.len()

In [28]:
#Y volvemos a obtener el promedio y la mediana
sueldos_data.groupby('trabajo_de').agg({'cantidad_lenguajes': ['mean', 'median']                                })

cantidad_lenguajes       
                                               mean median
trabajo_de                                                
bi analyst / data analyst                  2.412903    2.0
data center operadores de as 400                NaN    NaN
data entry                                      NaN    NaN
data scientist / data engineer             2.976744    3.0
data visualization                         3.000000    3.0
data visualization engineer                2.000000    2.0